In [1]:
import h5py
#from torchvision import datasets, transforms
import torch
from tqdm.auto import tqdm
#import os
#import pandas as pd
import numpy as np
#import random
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter

ModuleNotFoundError: No module named 'h5py'

# Select device which is available

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on GPU")
else:
    device = torch.device("cpu")
    print("Running on CPU")

Running on GPU


# PointNet Architecture

In [3]:
import torch.nn as nn
import torch.nn.functional as F

In [4]:
class Tnet(nn.Module):
    def __init__(self,k):
        super().__init__()
        self.k = k
        
        self.conv1 = nn.Conv1d(in_channels = k,out_channels = 64,kernel_size=1) #k instead of 3
        self.conv2 = nn.Conv1d(in_channels = 64,out_channels = 128,kernel_size=1)
        self.conv3 = nn.Conv1d(in_channels = 128,out_channels = 1024,kernel_size=1)
        
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)
        
        self.fc1 = nn.Linear(1024,512)
        self.fc2 = nn.Linear(512,256)
        self.fc3 = nn.Linear(256,k*k)
        
    
    def forward(self,x):
        point_cloud_size = len(x)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        
        x = torch.max(x,2,keepdim=True)[0]
        x = x.view(-1,1024)

        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)
        
        #if self.k == 3:
        #    iden = torch.from_numpy(np.eye(self.k).flatten().astype(np.float32)).view(1,self.k*self.k).repeat(point_cloud_size,1).to(device)
        #elif self.k == 64:
        iden = torch.from_numpy(np.eye(self.k).flatten().astype(np.float32)).view(1,self.k*self.k).repeat(point_cloud_size,1).to(device)
        
        x = x + iden
        x = x.view(-1, self.k, self.k)
        return x
        

In [5]:
class InputTransform(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.tnet = Tnet(3).to(device)
        
    def forward(self,x):
        input_trans = self.tnet(x).to(device)
        x = x.transpose(2,1) #??
        x = torch.bmm(x,input_trans)
        x = x.transpose(2,1) #??
        return x

In [6]:
class FeatureTransform(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.tnet = Tnet(64)
        
    def forward(self,x):
        feature_transform = self.tnet(x)
        x = x.transpose(2,1) #??
        x = torch.bmm(x,feature_transform)
        x = x.transpose(2,1) #??
        return x

In [7]:
class Classification(nn.Module):
    def __init__(self,k=2):
        super().__init__()
    
        self.fc1 = nn.Linear(1024,512)
        self.fc2 = nn.Linear(512,256)
        self.fc3 = nn.Linear(256,k)
        
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(p=0.7)
    
    def forward(self,x):
        x = self.dropout(F.relu(self.bn1(self.fc1(x))))
        x = self.dropout(F.relu(self.bn2(self.fc2(x))))
        x = F.softmax(self.fc3(x),dim=1)
        return x

In [8]:
class PointNetCls(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.input_transform = InputTransform()
        self.feature_transform = FeatureTransform()
        
        self.conv1 = nn.Conv1d(3,64,1) #???? really value? Arcitecture says (64/64)
        self.conv2 = nn.Conv1d(64,128,1)
        self.conv3 = nn.Conv1d(128,1024,1) 
        
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        
        self.classification = Classification()
        
    def forward(self,x):
        n_points = x.shape[0] #n x 3
        
        x = self.input_transform(x) #Input Transform
        
        x = F.relu(self.bn1(self.conv1(x))) #MLP Shared(64,64)             ???? really value? Arcitecture says (64/64)
        
        #n x 64
        
        x = self.feature_transform(x) #Feature Transform
        
        point_feature = x #n x 64 (Point Features for segmentation (Maybe use later))
        
        x = F.relu(self.bn2(self.conv2(x))) #MLP 64 --> 128
        x = self.bn3(self.conv3(x)) #MLP 128 --> 1024
        
        x = torch.max(x, 2, keepdim=True)[0]
            
        x = x.view(-1,1024) #Global Feature (Missing Link to segment)
        
        x = self.classification(x) #MLP (512,256,k) + output scores // Classification
        
        return x

In [9]:
def number_of_pointclouds(group,data_path):
    with h5py.File(f'{data_path}/shuffled_pointcloud2_hdf5.h5','r') as hdf:
        n_pointclouds = len(list(hdf.get(f'{group}/PointClouds')))
        return n_pointclouds

In [10]:
def get_batch(index,batch_size,data_path,category_set):
    pointcloud_list = []
    label_list = []
    
    with h5py.File(f'{data_path}/shuffled_pointcloud2_hdf5.h5','r') as hdf:

        for pointcloud in list(hdf.get(f'{category_set}/PointClouds'))[index:index+batch_size]:
            array = np.array(hdf.get(f'{category_set}/PointClouds/{pointcloud}'))
            pointcloud_list.append(array)

        for label in list(hdf.get(f'{category_set}/Labels'))[index:index+batch_size]:
            array = np.array(hdf.get(f'{category_set}/Labels/{label}'))
            label_list.append(array)
     
    pointcloud_tensors = torch.Tensor(pointcloud_list).view(-1,3,POINTCLOUD_SIZE).to(device)
    label_tensors = torch.Tensor(label_list).to(device)

    return pointcloud_tensors, label_tensors

In [11]:
def get_validation_data(data_path):
    pointcloud_list = []
    label_list = []
    
    with h5py.File(f'{data_path}/shuffled_pointcloud2_hdf5.h5','r') as hdf:

        for pointcloud in list(hdf.get('Validation/PointClouds')):
            array = np.array(hdf.get(f'Validation/PointClouds/{pointcloud}'))
            pointcloud_list.append(array)

        for label in list(hdf.get('Validation/Labels')):
            array = np.array(hdf.get(f'Validation/Labels/{label}'))
            label_list.append(array)
     
    pointcloud_tensors = torch.Tensor(pointcloud_list).view(-1,3,POINTCLOUD_SIZE).to(device)
    label_tensors = torch.Tensor(label_list).to(device)

    return pointcloud_tensors, label_tensors

# Fitting

In [12]:
def train2(model,data,labels,loss_function,optimizer):
    model.train()
    
    predictions = model(data)
    loss = loss_function(predictions,labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.item()

In [13]:
def validate(model,data,labels,loss_function):
    model.eval()
    
    predictions = model(data)
    loss = loss_function(predictions,labels)
    
    return loss.item()

In [16]:
EPOCHS = 20
BATCH_SIZE = 5
POINTCLOUD_SIZE = 1600

data_path = 'C:/Users/Kasper/Desktop/Compressed/PointCloudv1/PointCloudv1'
writer = SummaryWriter()
model = PointNetCls().to(device)

n_pointclouds = number_of_pointclouds('Training',data_path)

optimizer = torch.optim.Adam(model.parameters(),lr=0.00001,betas=(0.9, 0.999)) #MOMENTUM 0.9
loss_function = nn.MSELoss()

for epoch in range(EPOCHS):
    for i in tqdm(range(0,n_pointclouds,BATCH_SIZE)):
        train_X, train_y = get_batch(i,BATCH_SIZE,data_path,'Training')
        #val_X, val_y = get_batch(i,BATCH_SIZE,data_path,'Validation')
        val_X, val_y = get_validation_data(data_path)
        
        train_loss = train2(model,train_X,train_y,loss_function,optimizer)
        val_loss = validate(model,val_X,val_y,loss_function)
    
    print(f'Epoch: {epoch} | Train Loss: {train_loss} | Validation Loss: {validation_loss} ')
    writer.add_scalar('Loss/train', train_loss, epoch)
    writer.add_scalar('Loss/validation', val_loss, epoch)


Epoch: 0



Epoch: 1



Epoch: 2



Epoch: 3



Epoch: 4


...


Epoch: 15



Epoch: 16



Epoch: 17



Epoch: 18



Epoch: 19


In [ ]:
EPOCHS = 50
BATCH_SIZE = 15

writer = SummaryWriter()


def train(net,data_path):
    loss_list = []
    optimizer = torch.optim.Adam(net.parameters(),lr=0.00001,betas=(0.9, 0.999)) #MOMENTUM 0.9
    loss_function = nn.MSELoss()
    
    for epoch in range(EPOCHS):
        for i in tqdm(range(0,number_of_pointclouds('Training',data_path),BATCH_SIZE)):
            batch_X, batch_y = get_batch(i,BATCH_SIZE,data_path)

            net.zero_grad()
            
            outputs = net(batch_X)  
            loss = loss_function(outputs,batch_y)
            
            loss.backward()
            loss_list.append(loss)
            
            optimizer.step()
    
        #writer.add_graph(net,outputs)
        writer.add_scalar('Loss/train', loss, epoch)
        #print(f'Epoch: {epoch}. Loss {loss}.')
    
    plt.plot(loss_list)
            
            

In [ ]:
data_path = 'C:/Users/Kasper/Desktop/Compressed/PointCloudv1'
pointnet_cls = PointNetCls().to(device)
train(pointnet_cls,data_path)

# Testing

In [ ]:
def get_testing_data(data_path):
    pointcloud_list = []
    label_list = []
    
    with h5py.File(f'{data_path}/shuffled_pointcloud2_hdf5.h5','r') as hdf:

        for pointcloud in list(hdf.get('Validation/PointClouds')):
            array = np.array(hdf.get(f'Validation/PointClouds/{pointcloud}'))
            pointcloud_list.append(array)

        for label in list(hdf.get('Validation/Labels')):
            array = np.array(hdf.get(f'Validation/Labels/{label}'))
            label_list.append(array)
     
    pointcloud_tensors = torch.Tensor(pointcloud_list).view(-1,3,POINTCLOUD_SIZE).to(device)
    label_tensors = torch.Tensor(label_list).to(device)

    return pointcloud_tensors, label_tensors

In [ ]:
def predict(net,data_path):
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for i in tqdm(range(number_of_pointclouds('Testing',data_path))):
            test_X,test_y = get_testing_data(data_path)
            true_class = torch.argmax(test_y[i]).to(device)
            net_out = net(test_X[i].view(-1,3,POINTCLOUD_SIZE)).to(device)[0]
            predicted_class = torch.argmax(net_out)
            print(f'Real Class: {true_class}')
            print(f'Classification: {net_out}')
            print(f'Predicted Class: {predicted_class} \n')
            

            if predicted_class == true_class:
                correct += 1
            total += 1
    print("Accuracy: ",round(correct/total,3))
    
predict(pointnet_cls,data_path)